# Differential Privacy
The point of this notebook is to explore how we can introduce the concept of differential privacy to our dataset. This means adding statistical noise to sensitive data fields so it's harder to reverse engineer individuals to whom the data belongs. Local differential privacy is similar, it just specifies that we add this statisical noise right at the beginning when the data is being recorded.

Ultimately since the statistical noise is something like Laplace normal, or regular normal, the overall results don't change, but individuals remain secret. The amount of noise we add is based on the epsilon. 

For a binary class like Sex, the epsilon goes into the probability with which we flip the class. With Age, it's a simpler addition of Laplace noise.

In [1]:
import numpy as np
import pandas as pd
from aif360.datasets import AdultDataset

# load dataset
dataset = AdultDataset()
df = pd.DataFrame(dataset.features, columns=dataset.feature_names)
df['income'] = dataset.labels.ravel()

# DON'T binarize age
# age_median = df['age'].median()
# df['age_binary'] = (df['age'] > age_median).astype(int)
df['age'] = df['age'].astype(int)

# clean binary sex col as well
df['sex_binary'] = df['sex'].astype(int)
df[['age', 'sex_binary', 'income']].head(30)


,age,sex_binary,income
0,25,1,0.0
1,38,1,0.0
2,28,1,1.0
3,44,1,1.0
4,34,1,0.0
5,63,1,1.0
6,24,0,0.0
7,55,1,0.0
8,65,1,1.0
9,36,1,0.0


Now that we have binarized Age + Sex, we can have 4 such categories. So we can bucketise them.

In [2]:
# show sample of the dataset
df.sample(20)

,age,education-num,race,sex,capital-gain,capital-loss,hours-per-week,workclass=Federal-gov,workclass=Local-gov,workclass=Private,...,native-country=Scotland,native-country=South,native-country=Taiwan,native-country=Thailand,native-country=Trinadad&Tobago,native-country=United-States,native-country=Vietnam,native-country=Yugoslavia,income,sex_binary
38614,53,9.0,1.0,1.0,0.0,0.0,40.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1
42756,25,9.0,1.0,1.0,0.0,0.0,75.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1
11629,21,10.0,1.0,0.0,0.0,0.0,40.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
29880,68,9.0,1.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
1989,19,10.0,1.0,0.0,0.0,0.0,20.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
7371,33,10.0,1.0,0.0,0.0,0.0,40.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
19381,46,2.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
15256,37,9.0,1.0,1.0,0.0,0.0,40.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1
27748,27,13.0,0.0,0.0,0.0,0.0,33.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
42468,66,9.0,1.0,1.0,3432.0,0.0,20.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1


In [3]:
# buckets: 
# 0 = young & sex=0
# 1 = young & sex=1
# 2 = old & sex=0
# 3 = old & sex=1
# df['age_sex_cat'] = df['age'] * 2 + df['sex_binary']

# df[['age', 'age_binary', 'sex', 'sex_binary', 'age_sex_cat']].sample(10)

In [4]:
# true cross-tabulation to show the real counts before adding noise
true_ct = pd.crosstab(df['age'], df['sex'])
print("True Age x Sex cross-tab:")
print(true_ct)

True Age x Sex cross-tab:
sex  0.0  1.0
age          
17   245  248
18   342  353
19   422  442
20   429  487
21   416  519
..   ...  ...
86     1    0
87     0    1
88     1    4
89     0    1
90    12   34

[74 rows x 2 columns]


## Introducing a local differential privacy (DP) method using epsilon
The method will randomize responses based on the epsilon passed.
Reference: https://programming-dp.com/chapter13.html

In [5]:
def dp_randomized_response(categories, epsilon, k=4):
    # this function implements randomized response mechanism
    # which takes in categorical data in {0, .., k-1} and returns privatized "reports", which are the final outputs
    # that satisfy epsilon-differential privacy for each individual.
    # epsilon is the privacy parameter, so the higher it is, the less noise we add
    # The point is to "hide" the true category of each individual while still allowing overall statistics to be done without too much error

    # returns the following:
    # reports: privatized reports after applying DP randomized response
    # p: probability of telling the truth
    # q: probability of reporting some other value
    categories = np.asarray(categories, dtype=int)
    n = len(categories)
    
    # RR probabilities
    exp_eps = np.exp(epsilon)
    p = exp_eps / (exp_eps + k - 1) # prob of telling the truth
    q = (1.0 - p) / (k - 1) # prob of reporting some other value
    
    reports = np.empty_like(categories)
    u = np.random.rand(n) # pick uniform random numbers for each individual
    same = (u < p) # where we report the true category
    
    reports[same] = categories[same]
    
    # now see how many we need to flip, for those not reporting truth, sample from the other k-1 categories
    num_flip = np.sum(~same)
    # print(num_flip)
    if num_flip > 0:
        true_vals = categories[~same]
        # sample from other categories
        alt = np.random.randint(0, k-1, size=num_flip)
        # print((alt >= true_vals))
        alt += (alt >= true_vals).astype(int) # shift up by 1 if we hit the true value TODO check if this works!
        reports[~same] = alt # set alt values in the reports where we are not telling the truth
    
    return reports, p, q


In [6]:
def add_laplace_noise_to_age(df, epsilon, sensitivity):
    # apply the (local) Laplace mechanism to the 'age' attribute.
    # so for each individual:
    #    age_noisy = age_true + Laplace(0, b)
    # where b = sensitivity / epsilon.

    # epsilon: dictates level of privacy (higher epsilon = less noise = less privacy, vice versa)
    # sensitivity: maximum change in age (in "units") for one person, for this maybe we can treat it as 1 year?
    # clip: clip back to observed [min_age, max_age] if True.
    rng = np.random.default_rng(seed=23)

    df_priv = df.copy()
    ages = df_priv["age"].astype(float).to_numpy()

    b = sensitivity / epsilon # Laplace scale
    noise = rng.laplace(loc=0.0, scale=b, size=len(ages))
    ages_noisy = ages + noise

    df_priv["age_ldp"] = ages_noisy

    # clip to observed min/max age
    min_age = df["age"].min()
    max_age = df["age"].max()
    df_priv["age_ldp"] = df_priv["age_ldp"].clip(min_age, max_age)

    # round ages to integers
    df_priv["age_ldp"] = df_priv["age_ldp"].round().astype(int)

    return df_priv, b


In [7]:
def estimate_counts_from_dp(reports, p, q, k=4):
    # this function takes in the privatized reports from DP randomized response
    # and estimates the true counts for each category
    # we calculate the observed frequencies after noise, and then invert the process to get estimated "true" probabilities
    # this will tell us how many people are in each category approximately and we can compare with true counts to see the error introduced by DP method
    reports = np.asarray(reports, dtype=int)
    N = len(reports)
    counts_noisy = np.bincount(reports).astype(float)
    
    # calc p and q
    # exp_eps = np.exp(epsilon)
    # p = exp_eps / (exp_eps + k - 1)
    # q = (1.0 - p) / (k - 1)
    
    # freq_hat is the observed frequency after noise
    # probs_hat is the estimated true probabilities
    freq_hat = counts_noisy / N
    probs_hat = (freq_hat - q) / (p - q)
    
    # hack: clip to [0, 1] so that we don't get invalid probabilities, don't know if this is the right way to do it or if something went wrong earlier
    probs_hat = np.clip(probs_hat, 0, 1)
    counts_est = probs_hat * N
    return counts_noisy, counts_est


In [8]:

epsilons = [0.1, 0.3, 0.5, 0.8, 1.0, 1.5, 2.0, 5.0]
ages_true = df["age"].astype(float).to_numpy()
lap_results = []

for eps in epsilons:
    df_priv_eps, b = add_laplace_noise_to_age(df, epsilon=eps, sensitivity=2.0)
    ages_noisy = df_priv_eps["age"].to_numpy()

    # compute abs error and MSE
    abs_err = np.abs(ages_noisy - ages_true)
    mse = np.mean((ages_noisy - ages_true) ** 2)

    lap_results.append({
        "epsilon": eps,
        "scale_b": b,
        "mean_abs_error": abs_err.mean(),
        "mse": mse,
    })

pd.DataFrame(lap_results)

,epsilon,scale_b,mean_abs_error,mse
0,0.1,20.000000,0.0,0.0
1,0.3,6.666667,0.0,0.0
2,0.5,4.000000,0.0,0.0
3,0.8,2.500000,0.0,0.0
4,1.0,2.000000,0.0,0.0
5,1.5,1.333333,0.0,0.0
6,2.0,1.000000,0.0,0.0
7,5.0,0.400000,0.0,0.0


In [9]:
# build the privatized dataset with Laplace-noisy age

epsilon = 1.0  # for age we can have a higher epsilon since it's numeric
df_priv, b = add_laplace_noise_to_age(df, epsilon=epsilon, sensitivity=2.0)

df_priv = df_priv.copy()

epsilon = 1.0 # for sex, lower is better since it's categorical
# make sex as a dp_randomized_response
df_priv['sex_binary_ldp'], p, q = dp_randomized_response(df_priv['sex_binary'], epsilon=1.0, k=2)

df_priv[["age", "age_ldp", "sex_binary", "sex_binary_ldp", "income"]].head(20)


,age,age_ldp,sex_binary,sex_binary_ldp,income
0,25,26,1,1,0.0
1,38,39,1,1,0.0
2,28,25,1,1,1.0
3,44,41,1,1,1.0
4,34,35,1,1,0.0
5,63,65,1,1,1.0
6,24,22,0,0,0.0
7,55,53,1,1,0.0
8,65,66,1,1,1.0
9,36,36,1,1,0.0


Epsilon=1 seems like a reasonable value? But we can explore other values as well and see how the classifier fairs. If performance dips too much, or is unaffected, consider increasing epsilon to 2 or 2.5.

In [10]:
# drop original sensitive columns to be a purist
df_priv = df_priv.drop(columns=['age', 'sex', 'sex_binary'])

In [11]:
# this shows how many young/old and male/female individuals are there in the privatized dataset now
private_ct = pd.crosstab(df_priv['age_ldp'], df_priv['sex_binary_ldp'])
print("Private dataset counts:")
print(private_ct)

# compared to original true counts
true_ct = pd.crosstab(df['age'], df['sex_binary'])
print("\nOriginal dataset counts:")
print(true_ct)

Private dataset counts:
sex_binary_ldp    0    1
age_ldp                 
17              560  566
18              297  310
19              368  349
20              419  444
21              433  493
...             ...  ...
86                1    5
87                2    7
88                4    3
89                8    8
90               11   12

[74 rows x 2 columns]

Original dataset counts:
sex_binary    0    1
age                 
17          245  248
18          342  353
19          422  442
20          429  487
21          416  519
..          ...  ...
86            1    0
87            0    1
88            1    4
89            0    1
90           12   34

[74 rows x 2 columns]


In [12]:
# show a sample of the privatized dataset
df_priv.sample(20)

,education-num,race,capital-gain,capital-loss,hours-per-week,workclass=Federal-gov,workclass=Local-gov,workclass=Private,workclass=Self-emp-inc,workclass=Self-emp-not-inc,...,native-country=South,native-country=Taiwan,native-country=Thailand,native-country=Trinadad&Tobago,native-country=United-States,native-country=Vietnam,native-country=Yugoslavia,income,age_ldp,sex_binary_ldp
33896,9.0,1.0,0.0,0.0,40.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,55,1
43943,13.0,1.0,0.0,0.0,40.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,24,1
28831,9.0,1.0,0.0,0.0,50.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,40,0
16590,7.0,1.0,0.0,0.0,45.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,37,1
18146,6.0,0.0,0.0,0.0,32.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,28,1
31422,9.0,1.0,0.0,0.0,40.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,26,0
15901,9.0,0.0,0.0,0.0,40.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,22,1
44560,13.0,1.0,0.0,0.0,42.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,29,0
6885,10.0,0.0,0.0,0.0,40.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,42,0
32763,11.0,1.0,0.0,0.0,40.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,37,0


Now we can use the `df_priv` to train our classifier the same way we trained it with the full dataset.

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

In [14]:
# Drop the original age column and use the binarized version
X = df_priv.drop(['income'], axis=1)  # all the features that the model will use for prediction
y = df_priv['income'] # what we are trying to predict

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"Feature names: {list(X.columns)}")

Features shape: (45222, 98)
Target shape: (45222,)
Feature names: ['education-num', 'race', 'capital-gain', 'capital-loss', 'hours-per-week', 'workclass=Federal-gov', 'workclass=Local-gov', 'workclass=Private', 'workclass=Self-emp-inc', 'workclass=Self-emp-not-inc', 'workclass=State-gov', 'workclass=Without-pay', 'education=10th', 'education=11th', 'education=12th', 'education=1st-4th', 'education=5th-6th', 'education=7th-8th', 'education=9th', 'education=Assoc-acdm', 'education=Assoc-voc', 'education=Bachelors', 'education=Doctorate', 'education=HS-grad', 'education=Masters', 'education=Preschool', 'education=Prof-school', 'education=Some-college', 'marital-status=Divorced', 'marital-status=Married-AF-spouse', 'marital-status=Married-civ-spouse', 'marital-status=Married-spouse-absent', 'marital-status=Never-married', 'marital-status=Separated', 'marital-status=Widowed', 'occupation=Adm-clerical', 'occupation=Armed-Forces', 'occupation=Craft-repair', 'occupation=Exec-managerial', 'occu

In [15]:
# Split: 80% train, 20% test
# random_state : Ensures reproducibility
# stratify : Ensures class distribution is preserved in splits
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set size: {X_train.shape[0]} ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Test set size: {X_test.shape[0]} ({X_test.shape[0]/len(X)*100:.1f}%)")

print(f"\nClass distribution in training set:")
print(y_train.value_counts(normalize=True))
print(f"\nClass distribution in test set:")
print(y_test.value_counts(normalize=True))

Training set size: 36177 (80.0%)
Test set size: 9045 (20.0%)

Class distribution in training set:
income
0.0    0.752163
1.0    0.247837
Name: proportion, dtype: float64

Class distribution in test set:
income
0.0    0.752128
1.0    0.247872
Name: proportion, dtype: float64


In [16]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) # computes mean and std, then scales
X_test_scaled = scaler.transform(X_test) # not recomputing mean and std, just scaling

print("Features scaled using StandardScaler")
print(f"Training set scaled shape: {X_train_scaled.shape}")
print(f"Test set scaled shape: {X_test_scaled.shape}")

Features scaled using StandardScaler
Training set scaled shape: (36177, 98)
Test set scaled shape: (9045, 98)


In [17]:
lr_classifier = LogisticRegression(max_iter=1000)
lr_classifier.fit(X_train_scaled, y_train)
print("Logistic Regression trained")

Logistic Regression trained


In [18]:
# Predict on test set
y_test_pred = lr_classifier.predict(X_test_scaled)

# Calculate test metrics
test_accuracy = accuracy_score(y_test, y_test_pred)
test_precision = precision_score(y_test, y_test_pred)
test_recall = recall_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)

print("TEST SET PERFORMANCE")
print("="*50)
print(f"Accuracy:  {test_accuracy:.4f}")
print(f"Precision: {test_precision:.4f}")
print(f"Recall:    {test_recall:.4f}")
print(f"F1-Score:  {test_f1:.4f}")

print("\nDetailed Classification Report:")
print(classification_report(y_test, y_test_pred, target_names=['<=50K', '>50K']))

TEST SET PERFORMANCE
Accuracy:  0.8440
Precision: 0.7299
Recall:    0.5883
F1-Score:  0.6515

Detailed Classification Report:
              precision    recall  f1-score   support

       <=50K       0.87      0.93      0.90      6803
        >50K       0.73      0.59      0.65      2242

    accuracy                           0.84      9045
   macro avg       0.80      0.76      0.78      9045
weighted avg       0.84      0.84      0.84      9045



Comparing this to the metrics from our Classification.ipynb notebook, conclusion is thatt performing differential privacy with Laplace noise mechanism or Randomised Response both result in very similar results.